In [1]:
from functools import partial

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils import data
from torch.utils.data import random_split

import pytorch_lightning as pl # Lightning 主套件

from torchvision.datasets import MNIST
from torchvision import transforms as trans

import model_mnist


In [2]:
%rm -rf lightning_logs
BATCH_SIZE=1024
EPOCHS=2

In [3]:
## ---定義 Model---
net=nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )
## ---指定 loss function, optimizer---
loss_fn=F.cross_entropy
opt=partial(torch.optim.Adam,lr=0.003)

## ---放進model class---
model=model_mnist.Model(net,loss_fn,opt)

In [4]:
MNIST_train = MNIST("",train=True, download=True, transform=trans.ToTensor())
MNIST_test = MNIST("",train=False, download=True, transform=trans.ToTensor())
trainset, valset = random_split(MNIST_train, [55000, 5000])
train_loader = data.DataLoader(trainset, batch_size=32,num_workers=4)
val_loader = data.DataLoader(valset, batch_size=32,num_workers=4)
test_loader = data.DataLoader(MNIST_test, batch_size=32,num_workers=4)

/home/jovyan/.local/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
trainer = pl.Trainer(gpus=2,accelerator="dp",max_epochs=EPOCHS)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [7]:
trainer.fit(model,train_loader,val_loader)
trainer.test(model,test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 101 K 
------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.407     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/deprecate/deprecation.py:115: LightningDeprecationWarning: The `accuracy` was deprecated since v1.3.0 in favor of `torchmetrics.functional.classification.accuracy.accuracy`. It will be removed in v1.5.0.
  stream(template_mgs % msg_args)


Training: -1it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1046: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Testing: 0it [00:00, ?it/s]

KeyboardInterrupt: 

In [8]:
model.log_dict()